#### Convert the data and labels

### Classifying Urban sounds using Deep Learning

#### Visualizing Audio

#### Load Dataset

In [4]:
import librosa
import numpy as np

def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

In [5]:
# Load various imports 
import pandas as pd
import os
import librosa

# Set the path to the full UrbanSound dataset 
fulldatasetpath = './model/barkend/audio/'
metadata = pd.read_csv('./model/barkend/metadata/barkend.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    file_name = os.path.join(
        os.path.abspath(fulldatasetpath),
        str(row["fold"]) + '/',
        str(row["slice_file_name"])
    )
    
    class_label = row["class"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class'])

print('Finished feature extraction from ', len(featuresdf), ' files')

Finished feature extraction from  326  files


In [6]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import numpy as np

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
class_list = featuresdf['class']
y = np.array(class_list.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

Using TensorFlow backend.
/home/scardin/anaconda3/envs/dog-bark/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/scardin/anaconda3/envs/dog-bark/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/scardin/anaconda3/envs/dog-bark/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/scard

#### Split the dataset

In [7]:
# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

### Training and Evaluation

In [8]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

#### Compile the model

In [9]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [10]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                

#### Training

In [11]:
from keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='model/weights.hdf5',
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)

duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 260 samples, validate on 66 samples
Epoch 1/100
260/260 [==============================] - 0s 563us/step - loss: 44.6552 - accuracy: 0.2154 - val_loss: 18.2946 - val_accuracy: 0.3788

Epoch 00001: val_loss improved from inf to 18.29457, saving model to model/weights.hdf5
Epoch 2/100
260/260 [==============================] - 0s 82us/step - loss: 29.0559 - accuracy: 0.3077 - val_loss: 5.6289 - val_accuracy: 0.4394

Epoch 00002: val_loss improved from 18.29457 to 5.62894, saving model to model/weights.hdf5
Epoch 3/100
260/260 [==============================] - 0s 81us/step - loss: 20.3899 - accuracy: 0.3846 - val_loss: 3.5326 - val_accuracy: 0.5909

Epoch 00003: val_loss improved from 5.62894 to 3.53263, saving model to model/weights.hdf5
Epoch 4/100
260/260 [==============================] - 0s 79us/step - loss: 17.5714 - accuracy: 0.3269 - val_loss: 2.7255 - val_accuracy: 0.7727

Epoch 00004: val_loss improved from 3.53263 to 2.72551, saving model to model/weights.hdf5
Epoch 5


Epoch 00039: val_loss did not improve from 0.23100
Epoch 40/100
260/260 [==============================] - 0s 69us/step - loss: 0.6617 - accuracy: 0.8154 - val_loss: 0.2369 - val_accuracy: 0.9242

Epoch 00040: val_loss did not improve from 0.23100
Epoch 41/100
260/260 [==============================] - 0s 71us/step - loss: 0.5273 - accuracy: 0.8308 - val_loss: 0.2320 - val_accuracy: 0.9242

Epoch 00041: val_loss did not improve from 0.23100
Epoch 42/100
260/260 [==============================] - 0s 73us/step - loss: 0.5807 - accuracy: 0.8308 - val_loss: 0.2290 - val_accuracy: 0.9242

Epoch 00042: val_loss improved from 0.23100 to 0.22898, saving model to model/weights.hdf5
Epoch 43/100
260/260 [==============================] - 0s 70us/step - loss: 0.5193 - accuracy: 0.8615 - val_loss: 0.2537 - val_accuracy: 0.9091

Epoch 00043: val_loss did not improve from 0.22898
Epoch 44/100
260/260 [==============================] - 0s 73us/step - loss: 0.4902 - accuracy: 0.8538 - val_loss: 0.284

Epoch 79/100
260/260 [==============================] - 0s 73us/step - loss: 0.3862 - accuracy: 0.8846 - val_loss: 0.1273 - val_accuracy: 0.9697

Epoch 00079: val_loss did not improve from 0.12512
Epoch 80/100
260/260 [==============================] - 0s 71us/step - loss: 0.3194 - accuracy: 0.9000 - val_loss: 0.1641 - val_accuracy: 0.9545

Epoch 00080: val_loss did not improve from 0.12512
Epoch 81/100
260/260 [==============================] - 0s 70us/step - loss: 0.2676 - accuracy: 0.8885 - val_loss: 0.1881 - val_accuracy: 0.9545

Epoch 00081: val_loss did not improve from 0.12512
Epoch 82/100
260/260 [==============================] - 0s 72us/step - loss: 0.2310 - accuracy: 0.8962 - val_loss: 0.1683 - val_accuracy: 0.9394

Epoch 00082: val_loss did not improve from 0.12512
Epoch 83/100
260/260 [==============================] - 0s 73us/step - loss: 0.2025 - accuracy: 0.9231 - val_loss: 0.1386 - val_accuracy: 0.9394

Epoch 00083: val_loss did not improve from 0.12512
Epoch 84/100
26

#### Test the model

In [12]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9653846025466919
Testing Accuracy:  0.9696969985961914


#### Predictions

In [13]:
import librosa
import numpy as np

def extract_feature(file_name):
   
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    return np.array([mfccsscaled])

def print_prediction(file_name):
    prediction_feature = extract_feature(file_name)

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector)
    print("The predicted class is:", predicted_class[0], '\n')

    predicted_proba_vector = model.predict_proba(prediction_feature)
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)):
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f'))

#### Validation

In [17]:
# Class: Dog Bark

filename = './validation/chainsaw.wav' 
print_prediction(filename)

The predicted class is: chainsaw 

birds_chirping 		 :  0.00000000000018340632289802483168
chainsaw 		 :  0.99999976158142089843750000000000
dog_barking 		 :  0.00000007591842177134822122752666
engine 		 :  0.00000010636663461127682239748538
many_dogs_barking 		 :  0.00000000147808032480156725796405
people_talking 		 :  0.00000000000000992570232954828949
siren 		 :  0.00000000084233253705434663061169


In [15]:
import os

import tensorflow as tf
from tensorflow import keras

model.save('../barkend-ml-serving/model/barkend.hdf5')